In [2]:
from eventregistry import *
from newspaper import Article
from threading import Thread, Lock
import numpy as np
import pandas as pd
import random

# Print a list of recently added articles mentioning entered words
api_key = 'eda39267-9017-481a-860d-0b565c6d8bf3'
er = EventRegistry(apiKey = api_key)

global_df = pd.DataFrame()
mutex = Lock()

def get_articles(keywords): 
    global global_df
    q = QueryArticlesIter(keywords=QueryItems.AND(keywords))
    q.setRequestedResult(RequestArticlesInfo(count= 199, sortBy="sourceImportance"))
    print keywords
    
    x = 0
    
    local_df = pd.DataFrame()
    
    res = er.execQuery(q)
    for article in res['articles']['results']:
        data = {
            'source': article['source']['title'].encode('utf-8'),
#             'title' : article['title'].encode('utf-8'),
            'url' : article['url'].encode('utf-8'),
            'text' : article['body'].encode('utf-8')
        }
        local_df = pd.concat([local_df, pd.DataFrame(data,index=[x])])
        x += 1
        
    mutex.acquire()
    try:
        global_df = pd.concat([global_df,local_df])
    finally:
        mutex.release()

def get_search_params(keywords):
    search_params = []
    while len(keywords) != 0:
        # Randomly select 3 words
        rm = random.sample(keywords,3)
        # add the list of 3 words to the searchable list
        search_params.append(rm)
        # remove words from the list
        for word in rm:
            keywords.remove(word)

        # put 1 or 2 random words back
        # if 3 words left just append to search_params
        if len(keywords) is 3:
            search_params.append(keywords)
            keywords = []
        # if no words left just exit
        elif len(keywords) is 0:
            keywords = []
        # if 1 word left, append 2 and search_params
        elif len(keywords) is 1:
            keywords.append(random.sample(rm,2)[0:2])
        else:
            keywords.append(random.sample(rm,1)[0])
    return search_params

def get_keywords(user_url):
    url = user_url.decode('utf-8')
    article = Article(url)
    article.download()
    article.parse()
    article.nlp()

    keywords = article.keywords
    kl = []
    for word in keywords:
        kl.append(word.encode('utf-8'))
    return kl

class myThread(threading.Thread):
    def __init__(self, query):
        threading.Thread.__init__(self)
        self.query = query

    def run(self):
        get_articles(self.query)
        
def web_scrape(url):
    global global_df
    kl = get_keywords(url)
    params = get_search_params(kl)

    index = 0
    threads = []
    
    for query in params:
        threads.append(myThread(query))
        threads[index].start()
        index += 1
    for thread in threads:
        thread.join()

    global_df = global_df.reset_index(drop=True)
#     df.to_json(orient='index')
    global_df.to_csv('articles.csv')
    print global_df

def main():
    url = 'http://abcnews.go.com/US/wireStory/hurricanes-teach-us-ap-finds-fast-coastal-growth-49893843'
    web_scrape(url)

main()

using user provided API key for making requests
Event Registry host: http://eventregistry.org
['counties', 'continues', 'carolina']
['hurricanes', 'south', 'growth']
['growth', 'storms', 'counties']
['past', 'despite', 'storms']
['coastal', 'flooding', 'coastline']
['development', 'florida', 'coastline']
['lessons', 'irma', 'despite']
                      source  \
0         The New York Times   
1         The New York Times   
2               The Guardian   
3                Mail Online   
4                Mail Online   
5                Mail Online   
6                Mail Online   
7                Mail Online   
8                Mail Online   
9                Mail Online   
10               Mail Online   
11           Washington Post   
12           Washington Post   
13           Washington Post   
14           Washington Post   
15           Washington Post   
16           Washington Post   
17                 USA Today   
18                 USA Today   
19          The Daily C

In [41]:
df = pd.read_csv('articles.csv')
df

,Unnamed: 0,source,text,url
0,0,BBC News,"More than 1,000 people have been rescued in fl...",http://www.bbc.co.uk/news/world-us-canada-4106...
1,1,Mail Online,Miami was thrown a lifeline on Saturday as Hur...,http://www.dailymail.co.uk/~/article-4867972/i...
2,2,Mail Online,LOS ANGELES (AP) - Wildfires forced thousands ...,http://www.dailymail.co.uk/~/article-4849838/i...
3,3,Mail Online,The northern eyewall of Hurricane Irma has str...,http://www.dailymail.co.uk/~/article-4869944/i...
4,4,Mail Online,Rising sea levels and fierce storms have faile...,http://www.dailymail.co.uk/~/article-4890664/i...
5,5,Mail Online,Explosions that rocked a Texas chemical plant ...,http://www.dailymail.co.uk/~/article-4843464/i...
6,6,BBC News,As the clean-up operation continues in the flo...,http://www.bbc.co.uk/news/uk-northern-ireland-...
7,7,Mail Online,"By Bryan Sims\n\nHOUSTON, Sept 11 (Reuters) - ...",http://www.dailymail.co.uk/~/article-4873340/i...
8,8,Mail Online,The largest evacuation in US history is underw...,http://www.dailymail.co.uk/~/article-4867208/i...
9,9,Mail Online,Cost from Harvey for southeast Texas is alread...,http://www.dailymail.co.uk/~/article-4841102/i...


In [38]:
print arts

                  source                                               text  \
0                   test                                            testing   
0     The New York Times  FAIR BLUFF, N.C. -- The flooding is long gone,...   
1     The New York Times  WASHINGTON -- As Hurricane Irma rampaged throu...   
2     The New York Times  CLEWISTON, Fla. -- Vicious hurricanes all in a...   
3     The New York Times  The White House says Trump spoke to the offici...   
4     The New York Times  But even after the storm illustrated the city'...   
5     The New York Times  A hurricane watch was issued for Antigua and B...   
6     The New York Times  About 4 a.m. local time, the storm was 40 mile...   
7     The New York Times  "A lot of people died," Mr. Marte said of Harv...   
8     The New York Times  MEXICO CITY -- Sorry, we have too many trouble...   
9     The New York Times  If you're flying, call your airline to get a s...   
10    The New York Times  Over those years, many thi

In [33]:
arts['articles']['results'][0]['source']['title']
len(arts['articles']['results'])

199

In [4]:
df1 = pd.read_csv('100_articles.csv')
df1.columns = ['id','source','text',"""'title""" 'url']

bodies = df1.loc[:,['id','text']]
bodies.to_csv('bodies.csv')

# we want claim here, not title
# headlines = df1.loc[:,['id','title']]
# headlines.to_csv('headlines.csv')

# urls = df1.loc[:,['id','source','url']]
# urls.to_csv('url.csv')

In [15]:
claim = ['US coastal growth continues'] * 100
# claim = np.array(claim)
# # print type(claim)
# claims = df1.loc[:,['id']]
# claims = pd.concat([claims,pd.DataFrame(claim)])
claims = pd.DataFrame(claim)
claims.columns = 'claim'
print claims

                              0
0   US coastal growth continues
1   US coastal growth continues
2   US coastal growth continues
3   US coastal growth continues
4   US coastal growth continues
5   US coastal growth continues
6   US coastal growth continues
7   US coastal growth continues
8   US coastal growth continues
9   US coastal growth continues
10  US coastal growth continues
11  US coastal growth continues
12  US coastal growth continues
13  US coastal growth continues
14  US coastal growth continues
15  US coastal growth continues
16  US coastal growth continues
17  US coastal growth continues
18  US coastal growth continues
19  US coastal growth continues
20  US coastal growth continues
21  US coastal growth continues
22  US coastal growth continues
23  US coastal growth continues
24  US coastal growth continues
25  US coastal growth continues
26  US coastal growth continues
27  US coastal growth continues
28  US coastal growth continues
29  US coastal growth continues
..      

In [71]:
from threading import Thread, Lock

mutex = Lock()

aggregate_keywords = []

class myThread(threading.Thread):
    def __init__(self, url):
        threading.Thread.__init__(self)
        self.url = url

    def run(self):
        get_keywords(self.url)
        
def get_keywords(user_url):
    url = user_url.decode('utf-8')
    article = Article(url)
    article.download()
    article.parse()
    article.nlp()

    keywords = article.keywords
    kl = []
    for word in keywords:
        kl.append(word.encode('utf-8'))
    mutex.acquire()
    try:
        aggregate_keywords.append(kl)
        print 'try'
    finally:
        mutex.release()
        print 'finally'

urls = ['https://www.nytimes.com/2017/09/16/world/asia/kabul-green-zone-afghanistan.html?hp&action=click&pgtype=Homepage&clickSource=story-heading&module=first-column-region&region=top-news&WT.nav=top-news',
        'http://abcnews.go.com/US/wireStory/irma-shuttered-schools-add-stress-families-49894226?cid=clicksource_4380645_1_hero_headlines_headlines_hed',
       'http://www.cnn.com/2017/09/16/us/chicago-freezer-death/index.html',
       'http://www.cnn.com/2017/09/16/politics/russia-fires-on-us-backed-forces/index.html']

index = 0
threads = []
for url in urls: 
    threads.append(myThread(url))
    threads[index].start()
    index += 1
    print index

for thread in threads:
    thread.join()
print aggregate_keywords

        

1
2
3
4
try
finally
try
finally
try
finally
try
finally[['afghanistan', 'trump', 'zone', 'kabul', 'american', 'embassy', 'green', 'digging', 'mr', 'security', 'afghan', 'expands', 'decade'], ['deir', 'city', 'group', 'isis', 'syrian', 'coalition', 'strike', 'usbacked', 'rebel', 'forces', 'ezzor', 'russian', 'syria', 'sdf'], ['shown', 'death', 'chicago', 'footage', 'hotel', 'freezer', 'surveillance', 'jenkins', 'video', 'martin', 'cnn', 'seen', 'told'], ['selfsustaining', 'school', 'returning', 'keys', 'west', 'florida', 'county', 'officials', 'key', 'water', 'residents', 'south']]



In [3]:
import numpy as np
import pandas as pd
import io, json
dataframe = pd.read_csv('articles.csv')

dataframe = dataframe.loc[:100,['source','text','url']]
dataframe.to_csv('100_articles.csv')